In [1]:
import pandas as pd
import sqlalchemy, psycopg2, numpy as np
import os
from dotenv import load_dotenv

load_dotenv()




True

In [2]:
df = pd.read_csv(os.getenv('CSV_PATH'))
df.head() # se importó correctamente

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Exploración de tipo de datos

In [3]:
df.dtypes

# InvoiceDate es object, debería ser datetime.


InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [4]:
# Veo si existen nulos

df.isnull().sum()

#Existen nulos en descripción y customerID

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Limpieza de datos en pandas

In [5]:
# Se eliminan los valores nulos en CustomerID.

df = df.dropna(subset=['CustomerID'])

(df['CustomerID'].isna().sum() ) # No hay valores nulos en CustomerID ahora

np.int64(0)

In [6]:
# Verifico si existen duplicados en columnas a utilizar como IDs... columnas "StockCode", "InvoiceNo" y "CustomerID" tienen duplicados. Se eliminarán duplicados al crear tablas más adelante.

df.apply(lambda col: col.duplicated().any())

InvoiceNo      True
StockCode      True
Description    True
Quantity       True
InvoiceDate    True
UnitPrice      True
CustomerID     True
Country        True
dtype: bool

In [7]:
# Las unidades que están en 0 se considerarán como errores, por lo que las elimino del df

df = df[df['UnitPrice'] != 0]

In [8]:
# Las cantidades en negativo se consideran devoluciones, por lo que crearé una nueva columna con los valores de vueltos y dejaré como 0 los valores en negativo de la columna "Quantity", y viceversa.

df['refunds'] = np.where(df['Quantity'] < 0, df['Quantity'], 0)
df['Quantity'] = np.where(df['Quantity'] < 0, 0, df['Quantity'])

In [9]:
# En descripción existen valores con "?". Si tiene una descripción adicional al símbolo, se mantiene. Si no, se deja como 'Unknown'. Además, se rellenan valores en blanco como 'Unknown'.

df['Description'] = (
    df['Description']
    .str.strip()
    .replace(r'^\?+$', np.nan, regex=True)
    .replace("", np.nan)
    .str.replace("?", "", regex=False)
    .fillna("Unknown")
)


/* Separaré el df distintas tablas, considerando modelo entidad - relación tal como sigue:

PRODUCTS
- stockCode (productID)
- description

ORDERS
- order_id
- invoice_no (PK)
- invoice_date
- customer_id (FK)

ORDER_DETAILS
- order_id (PK)
- invoice_no (FK)
- stock_code (FK)
- quantity
- refunds
- unit_price

CUSTOMERS
- customer_id (PK)
- country

*/

In [10]:
# Tabla productos
products = pd.DataFrame(columns=['product_id', 'description'])
products['product_id'] = df['StockCode']
products['description'] = df['Description']

products = (
    products
    .drop_duplicates(subset=['product_id']) # Se eliminan duplicados
    .reset_index(drop=True)
)

# Tabla orders

orders = (
    df[['InvoiceNo', 'InvoiceDate', 'CustomerID']] # Se eliminan duplicados
    .drop_duplicates()
    .reset_index(drop=True)
)
# Clave surrogate
orders['order_id'] = orders.index + 1

# Reordenar columnas
orders = orders[['order_id', 'InvoiceNo', 'InvoiceDate', 'CustomerID']]

# Tabla order_details

order_details = df.copy()

# Propagar la clave surrogate (order_id)
order_details = order_details.merge(
    orders[['order_id', 'InvoiceNo', 'InvoiceDate', 'CustomerID']],
    on=['InvoiceNo', 'InvoiceDate', 'CustomerID'],
    how='left'
)

# Renombro columnas

orders= orders.rename(columns={
    'CustomerID': 'customer_id'
}
)

order_details= order_details.rename(columns={
    'CustomerID': 'customer_id'
}
)   

# Seleccionar columnas finales
order_details = order_details[[
    'order_id',
    'StockCode',
    'Quantity',
    'UnitPrice',
    'refunds'
]]

# Renombrar columnas
order_details = order_details.rename(columns={
    'StockCode': 'product_id',
    'Quantity': 'quantity',
    'UnitPrice': 'unit_price'
})


# Tabla customers. Acá no voy a eliminar duplicados, ya que en SQL crearé dim_customers con valores únicos y dejaré customers con los duplicados para cálculos globales.

customers = pd.DataFrame(columns=['customer_id', 'country'])
customers['customer_id'] = df['CustomerID']
customers['country'] = df['Country']



In [11]:
# --- OPTIMIZACIÓN DE TIPOS ANTES DE CARGA ---


# Castear precios y cantidades para asegurar precisión
order_details['unit_price'] = order_details['unit_price'].astype(float)
order_details['quantity'] = order_details['quantity'].astype(int)

# Castear CustomerID a int
customers['customer_id'] = customers['customer_id'].astype(int)

In [12]:
# Subo dfs creados a postgres para crear relaciones y realizar queries con insights

from sqlalchemy import create_engine, text

engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

# Creo lista de tablas a subir
tablas = {
    'products': products,
    'orders': orders,
    'order_details': order_details,
    'customers': customers
}

# Elimino tablas si existen

with engine.connect() as connection:
    for nombre_tabla in tablas.keys():
        connection.execute(
            text(f"DROP TABLE IF EXISTS {nombre_tabla} CASCADE")
        )
    connection.commit()

# Subo tablas a la base de datos

for nombre_tabla, df_tabla in tablas.items():
    df_tabla.to_sql(
        name=nombre_tabla,
        con=engine,
        if_exists='replace',
        index=False
    )